In [3]:
import nltk
#nltk.download()

In [4]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk import FreqDist
from textblob import TextBlob

#import contractions
#import inflect

In [5]:
violence_words = set(["asesinato", "asesino", "asesina", "matar", "mata", "mato", "apuñalar","apuñalo","pega","pego","apalizo","golpea","golpeo","agredio","agrede"])

In [6]:
print(violence_words)

{'asesinato', 'mato', 'apuñalar', 'agrede', 'asesino', 'apalizo', 'mata', 'agredio', 'pego', 'golpea', 'apuñalo', 'matar', 'pega', 'asesina', 'golpeo'}


In [7]:
stopWords = set(stopwords.words('spanish'))

In [8]:
print(stopWords)

{'sobre', 'tuvieses', 'vuestro', 'estuviéramos', 'tuyo', 'ellos', 'seamos', 'estuviesen', 'tú', 'me', 'eso', 'o', 'serán', 'se', 'hubierais', 'más', 'fuese', 'vuestras', 'tenida', 'sean', 'fuimos', 'hubieran', 'estadas', 'estarían', 'qué', 'era', 'tuvieran', 'vuestros', 'vosostros', 'eran', 'esta', 'un', 'otro', 'nuestros', 'estarás', 'contra', 'ellas', 'tenías', 'tenemos', 'e', 'tuviese', 'tuviéramos', 'son', 'tuvo', 'hubimos', 'cuando', 'tengas', 'cual', 'estéis', 'seré', 'habías', 'hubieseis', 'hubieras', 'sintiendo', 'estabais', 'hayáis', 'habíais', 'quien', 'habremos', 'mío', 'suya', 'estén', 'tenéis', 'fuesen', 'nos', 'en', 'he', 'hayas', 'tendrá', 'los', 'ante', 'hube', 'serás', 'fuiste', 'estado', 'seáis', 'fueran', 'tendrías', 'hubiera', 'estaban', 'habido', 'tendría', 'estuviese', 'entre', 'hayamos', 'estuvieses', 'ella', 'tuvieron', 'estaríamos', 'uno', 'estaremos', 'seas', 'habrás', 'mi', 'sentid', 'tendré', 'tuvimos', 'tuvieseis', 'poco', 'estamos', 'hay', 'la', 'estos', '

In [9]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [10]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [80]:
def get_headers(soup):
    headers = []
    date = []
    violence = []
    
    #esto el container de cada noticia
    container = soup.find_all('div',{'class','n1 fueraNoticia'})
    for item in container:
        headers.append(item.find('h2',{'class','titulo'}).text)
    for item in container:
        date.append(item.find('meta',itemprop='datePublished').get('content')[:10])
    
    
    if len(headers) == len(date):
        return pd.DataFrame({"HEADERS": headers, "DATE_PUBLISHED": date})
    else:
        return -1
    """
        #lst.append(header.text)
        date.append(item.find('meta',itemprop='datePublished').get('content')[:10]) 
        print(date)   
        #lst.append(date[:10])
        lst.append(False)
        #total_lst.append(lst)
    
    """
        
news_raw = get_headers(soup)
news_raw.head(20)

,HEADERS,DATE_PUBLISHED
0,"Un niño de 8 años, grave tras un ahogamiento e...",2019-06-26
1,"Una anciana, crítica al quedar atrapada en su ...",2019-06-26
2,"La auxiliar de enfermería de Alcalá, culpable ...",2019-06-26
3,Desarticulada una organización de trata de tra...,2019-06-26
4,Detenidos por acordar el asesinato de un hombr...,2019-06-26
5,Libertad para el presunto pederasta que simula...,2019-06-26
6,Agrede a su pareja en Hortaleza mordiéndole el...,2019-06-25
7,"El 'rey de la burundanga', arrestado de nuevo",2019-06-25
8,Detenido un pederasta que fingía ser agente se...,2019-06-25
9,Fallece un obrero en Vallecas al precipitarse ...,2019-06-24


In [59]:
# Get rid of lower-cases
def tolower (work_list):
    return [row.lower() for row in work_list]
    
news_raw = tolower(news_raw)
pprint(news_raw[:6])

AttributeError: 'int' object has no attribute 'lower'

In [11]:
# Get rid of accents
def clean_accent(work_list):
    return [row.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for row in work_list]

news_raw2 = clean_accent(news_raw)
pprint(news_raw2[:6])

['una anciana, critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala, culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos',
 'libertad para el presunto pederasta que simulaba ser agente secreto',
 'agrede a su pareja en hortaleza mordiendole el labio']


In [12]:
#Get rid of commas
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw3 = clean_any_character(news_raw2, ',')
pprint(news_raw3[:3])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales']


In [13]:
#Get rid of quotes
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw4 = clean_any_character(news_raw3, '\"')
pprint(news_raw4[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [14]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw5 = clean_any_character(news_raw4, '\'')
pprint(news_raw5[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [15]:
#Get rid of '
def clean_any_character(work_list, character):
    return [row.replace(character,'') for row in work_list]

news_raw6 = clean_any_character(news_raw5, ':')
pprint(news_raw6[:4])

['una anciana critica al quedar atrapada en su coche accidentado',
 'la auxiliar de enfermeria de alcala culpable de uno de los asesinatos',
 'desarticulada una organizacion de trata de transexuales',
 'detenidos por acordar el asesinato de un hombre para vender sus organos']


In [16]:
# Get each word / Tokenizar
def token(work_list):
    return [nltk.word_tokenize(row) for row in work_list]

token_news = token(news_raw6)
print(token_news[:6])

[['una', 'anciana', 'critica', 'al', 'quedar', 'atrapada', 'en', 'su', 'coche', 'accidentado'], ['la', 'auxiliar', 'de', 'enfermeria', 'de', 'alcala', 'culpable', 'de', 'uno', 'de', 'los', 'asesinatos'], ['desarticulada', 'una', 'organizacion', 'de', 'trata', 'de', 'transexuales'], ['detenidos', 'por', 'acordar', 'el', 'asesinato', 'de', 'un', 'hombre', 'para', 'vender', 'sus', 'organos'], ['libertad', 'para', 'el', 'presunto', 'pederasta', 'que', 'simulaba', 'ser', 'agente', 'secreto'], ['agrede', 'a', 'su', 'pareja', 'en', 'hortaleza', 'mordiendole', 'el', 'labio']]


In [17]:
def clean_stopwords(list_tokens, stp_words):
    [row.remove(item) for row in list_tokens for item in row if item in stp_words]
    return list_tokens
    
cleaned_data = clean_stopwords(token_news, stopWords)
pprint(cleaned_data[:6])

[['anciana', 'critica', 'quedar', 'atrapada', 'su', 'coche', 'accidentado'],
 ['auxiliar', 'enfermeria', 'alcala', 'culpable', 'uno', 'los', 'asesinatos'],
 ['desarticulada', 'organizacion', 'trata', 'transexuales'],
 ['detenidos', 'acordar', 'asesinato', 'un', 'hombre', 'vender', 'organos'],
 ['libertad',
  'el',
  'presunto',
  'pederasta',
  'simulaba',
  'ser',
  'agente',
  'secreto'],
 ['agrede', 'su', 'pareja', 'hortaleza', 'mordiendole', 'labio']]


In [28]:


#new = new + " enfado"
new = [' '.join(item) for item in cleaned_data]


for item in new:
    if TextBlob(item).detect_language() == 'es':
        print("1")
        analysis = TextBlob(str(item))
        item = analysis.translate(to='spanish')
        print(item)
    analysis = TextBlob(str(item))
    print(analysis.sentiment,"\n")




"""
new = ' '.join(cleaned_data[1])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")


new = ' '.join(cleaned_data[2])
print(new)
print(TextBlob(new).detect_language())
analysis = TextBlob(str(new))
print(analysis.sentiment,"\n")

"""
"""
new2 = analysis.translate(to='spanish')
analysis = TextBlob(str(new2))
print(analysis.sentiment)
print(analysis.tags)
#print(analysis.translate(to='spanish'))
print(dir(analysis))
"""


1
old lady criticizes getting caught her car accident
Sentiment(polarity=0.1, subjectivity=0.2) 

1
auxiliary nursing alcala guilty one the murders
Sentiment(polarity=-0.5, subjectivity=1.0) 

1
disjointed organization treats transsexuals
Sentiment(polarity=0.0, subjectivity=0.0) 

1
detainees accord murder a man sell organs
Sentiment(polarity=0.0, subjectivity=0.0) 

1
freedom the alleged pedophile pretended to be a secret agent
Sentiment(polarity=-0.25, subjectivity=0.39999999999999997) 

1
assaults her partner hortaleza biting her lip
Sentiment(polarity=0.0, subjectivity=0.0) 

1
king the burundanga arrested new
Sentiment(polarity=0.13636363636363635, subjectivity=0.45454545454545453) 

1
arrested pederasta pretended to be secret agent
Sentiment(polarity=-0.4, subjectivity=0.7) 

1
dies worker vallecas rush the elevator shaft
Sentiment(polarity=0.0, subjectivity=0.0) 

1
suffocated fire the palace chanting peak
Sentiment(polarity=0.0, subjectivity=0.0) 

1
burns shop nuts improved f

"\nnew2 = analysis.translate(to='spanish')\nanalysis = TextBlob(str(new2))\nprint(analysis.sentiment)\nprint(analysis.tags)\n#print(analysis.translate(to='spanish'))\nprint(dir(analysis))\n"

In [ ]:
def new_item(list_tokens):
    for index, row in enumerate(list_tokens):
        list_tokens.insert(index+1, [])
        list_tokens.insert(index+1, [])
    return list_tokens
 
print(new_item(cleaned_data))

In [ ]:
def violence_at_row(list_tokens,set_list):
    for row in list_tokens:
        for word in row:
            if word in violence_words:
                

In [ ]:
for word in cleaned_data:
    if 